In [ ]:
!pip install ipympl

In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 

%matplotlib widget

#inspired by https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

data = pd.read_excel('data.xlsx')
data.head()

X = data.iloc[:, 0:212].values #features
y = data.iloc[:, 212].values   #labels

np.save('X', X)
np.save('y', y)

In [7]:
data = pd.read_excel('data.xlsx') 

X = data.iloc[:, 0:212].values #features 
y = data.iloc[:, 212].values #labels

np.save('X', X) 
np.save('y', y)

### Load Data + Analysis

In [8]:
X = np.load('X.npy')
y = np.load('y.npy')

In [11]:
fig = plt.figure()
_ = plt.hist(y)
plt.xlabel('Decision')
plt.ylabel('# Occurence')
plt.suptitle('Histogram of Decision')
plt.show()

## On vérifie que la distribution est "logique" 
# et ça se vérifie car plus grande distribution en 0 (donc ne rien faire) ok mais attention 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
#on peut normalizer les données ici

X[:,0] = (X[:,0]-X[:,0].min())/np.max(X[:,0] - X[:,0].min()) #position en x [0,1]
X[:,1] = (X[:,1]-X[:,1].min())/np.max(X[:,1] - X[:,1].min()) #position en y [0,1]
X[:,2] = X[:,2]/360 #degre -> [0 1]

d_max = np.max(X[:, 4::4]) #on prend tous les éléments de l'array qui correspondent à la distance et on calcule la distance max
X[:, 4::4] = X[:, 4::4]/d_max #distance entre [0 1]

In [13]:
# Ici on va vérifier qu'on ait pas des lignes totatement vide (donc non pertinentes pour un algo ML)
id_sup = []
fig = plt.figure()
for i in range(X.shape[1]):
    if X[:,i].min() == X[:,i].max(): #si max = min => pas de variation
        plt.plot(X[:,i], label='ligne : '+str(i))
        id_sup.append(i)
plt.show()
print("Il y a {} lignes vides sur les {}.".format(len(id_sup), i+1)) #aïe aïe

X = np.delete(X, id_sup, axis=1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Il y a 56 lignes vides sur les 212.


In [14]:
X.shape

(4804, 156)

### Random Forest

In [15]:
# import
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [16]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
cf = RandomForestClassifier(n_jobs=-1)

In [17]:
# Number of trees in random forest
n_estimators = [10, 100, 1000]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [10, 100]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
params_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [18]:
grid_search = GridSearchCV(estimator=cf, param_grid=params_grid, cv=5)
grid_search.fit(X, y)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=-1,
                                              oob_score=False,
                                              random_

## Save Model

In [20]:
import pickle

# Save to file in the current working directory
pkl_filename = "pickle_model.pkl"

with open(pkl_filename, 'wb') as file:
    pickle.dump(grid_search, file)
